Use the word2vec model to convert the entire text to a vector. 

In [2]:
from gensim.models import Word2Vec
import pandas as pd

In [4]:
word2vec = Word2Vec.load("./word2vec/word2vec_200_fake_pii.bin")

In [8]:
string = "My email address is edwardcooper@gmail.com. I would love you to call me any time at 513-801-9094. Best,\
 Edward Cooper"

In [9]:
fake_pii = pd.read_csv("../data/text_with_pii_2018_12_21_04_32_47_173911.csv")

# clean the text in a sentence and return a sentence.
def clean_sentence(sentence):
    
    sentence_cleaned = [word.replace(",", "").replace(".", "")\
                        .lower() for word in sentence.split(" ")]
    return " ".join(sentence_cleaned)    

context_sentences = fake_pii["Text"].apply(clean_sentence)

In [10]:
tokenized_sentence_word_list = [sentence.split(" ")\
                                for sentence in context_sentences]

In [32]:
fake_pii.shape

(800000, 3)

In [87]:
fake_pii.iloc[300001,0]

'Impact region card technology indicate. Station cut rule Patricia Smith church Congress. There west parent write. Accept image region through group matter.'

In [90]:
result_text = context_sentences[300001].split()

result_text

['impact',
 'region',
 'card',
 'technology',
 'indicate',
 'station',
 'cut',
 'rule',
 'patricia',
 'smith',
 'church',
 'congress',
 'there',
 'west',
 'parent',
 'write',
 'accept',
 'image',
 'region',
 'through',
 'group',
 'matter']

In [91]:
word2vec.wv.doesnt_match(result_text)

'patricia'

In [80]:
context_sentences[0]

'ever watch television read measure almost thought shake take suite 971 program interest strong exist clear least like create most order court build organization'

In [92]:
word2vec.wv["patricia"]

array([ 0.36622727,  0.27836189, -0.37526613,  0.39204478,  0.54952252,
        0.42405748,  0.06321009,  0.39116091,  0.39493561,  0.42504805,
       -0.04260241, -0.32782874,  0.28235227,  0.20148884,  0.51235503,
        0.12329604,  0.39492026,  0.11548052,  0.48414013,  0.46749458,
       -0.09176376, -0.26889315,  0.00905478,  0.10210225,  0.82386851,
        0.22759764, -0.42818621, -0.46396604, -0.15678126, -0.07515994,
       -0.5305078 , -0.00258889, -0.46586043,  0.2786144 ,  0.21425438,
       -0.11656933, -0.00812311, -0.32174751, -0.38319516, -0.06494484,
        0.11989294,  0.08557372, -0.07803344,  0.65480614, -0.27052978,
       -0.231998  ,  0.34841606, -0.46566299,  0.54239953, -0.34700489,
       -0.48149049, -0.00473366,  0.16973442,  0.57294244,  0.29995993,
        0.11582996, -0.29220232,  0.03455056,  0.22629175, -0.00957377,
       -0.1343711 , -0.0322794 , -0.45557189, -0.25282791, -0.30939597,
       -0.42252561, -0.38229427,  0.70267814,  0.03212199, -0.56

In [93]:
fake_pii.head(10)

,Text,Labels,PII
0,Ever watch television read measure almost. Tho...,Address,Suite 971
1,Impact region card 37271 Adams Road Suite 438 ...,Address,37271 Adams Road Suite 438
2,0604 Mason Wall Throughout admit have science ...,Address,0604 Mason Wall
3,Able item bag my day impact continue. Apt. 633...,Address,Apt. 633
4,Inside perhaps Suite 744 else seven. Able seas...,Address,Suite 744
5,Feeling more reduce everything. Apt. 524 Creat...,Address,Apt. 524
6,Rule price recognize continue. Happy act agree...,Address,22471 Hannah Crossing
7,Suite 104 Some second dinner. Force past line ...,Address,Suite 104
8,"Large 930 Chad Ford Apt. 029 New Michaelmouth,...",Address,"930 Chad Ford Apt. 029 New Michaelmouth, KY 89394"
9,Future movement office. Thus building successf...,Address,"0874 Charles Forge Suite 659 Caitlinburgh, ME ..."


The next goal is to generate the embedding for the part of the PII. 

In [94]:
def find_part_pii(text, model, sep = " "):
    tokenized_text = text.split(sep)
    
    part_pii = model.wv.doesnt_match(tokenized_text)
    
    return part_pii    

In [97]:
import numpy as np
def get_pii2vec_matrix(pii_list, model):
    # set the matrix dimensions
    column_num = model.trainables.layer1_size
    row_num = len(pii_list)
    # initialized the matrix
    pii2vec_mat = np.zeros(row_num, column_num)
    # iterate through the pii_list and assign the vectors to matrix.
    for ith_pii in pii_list:
        pii2vec_mat[1,:] = model.wv[ith_pii]
    
    return pii2vec_mat

200